# Introduction à la Science des données


## Travail pratique 03 - Modélisation et analyse de performances

[Table des matières](#toc)

**Informations de groupe - Prénoms et noms complets:**

- Fehlmann Dylan
- Stelcher Stan
- ...

**Professeurs**: Carlos Peña et Stephan Robert

**Assistant(s)**: Thibault Schowing, Arthur Babey, Cédric Campos Carvalho

**Contact**: prenom.nom@heig-vd.ch ou de préférence via Teams 

### Modalités de rendu :

- **Date**: <span style="background-color:#eebbdd">01.12.2024, 23h55</span>


- **Travail par groupe** de 2 ou 3. 


- Une fois complété, rendez directement le notebook (fichier avec l'extension _.ipynb_) nommé correctement comme suit <span style="background-color:#eebbdd">"**TP3_ISD_SA2024_Nom1_Nom2(_Nom3).ipynb**"</span> en mettant les noms de famille de chaque membres du groupe (pour ceux avec plusieurs noms de famille, vous pouvez mettre juste le premier comme dans l'adresse email). Les TPs rendu avec un fichier mal nommé seront pénalisé !


- Mettez vos Prénoms et noms en entier ci-dessus. 


- Uploadez le fichier complété avant le délais sur Cyberlearn ou Teams selon les consignes données. 



### Déroulement et notation


<div class="alert alert-block alert-info">
    <b>Les questions</b> sont généralement indiquées en <b>gras</b>, en <span style="background-color:#AFEEEE">bleu</span> ou par une liste d'instructrions et les endroits où répondre sont indiqués par un "<i>Réponse:</i>" pour les réponses textuelles. Pour les réponses nécessitant d'écrire du code, les cellules ont déjà été crées et un commentaire indique où/quoi répondre. 
</div>


<div class="alert alert-block alert-info">
    <b>Notation: </b> Ce TP est noté sur 6 avec un total de  <span style="background-color:#eebbdd"><b>80</b></span> points. Les points sont indiqués pour chaques parties du travail pour un total de <span style="background-color:#eebbdd"><b>76</b></span> points  et <span style="background-color:#eebbdd"><b>4</b></span> points supplémentaires sont attribués au rendu du travail (format et nommage selon les consignes) et à la propreté (lisibilité et mise en page, tournure de phrase des réponses). 
</div>


<div class="alert alert-block alert-info">
<b>Corrections: </b> Ce notebook vous sera renvoyé via Cyberlearn/Teams ou un autre canal. Les informations principales concernant les corrections seront indiquées après chaque section (banière bleue) avec le nombre de points obtenus. Il est possible que des remarques concernant le code soient directement ajoutées dans celui-ci.
</div>

---


<div class="alert alert-block alert-success">
<b>Objectifs </b>
</div>

- Comprendre la modélisation avec un modèle simple: le modèle à base de règles
- Analyse des résultats avec la matrice de confusion
- Algorithme kNN
- Évaluation des performances avec Validation hold-out et N-Fold Crossvalidation. 

<div class="alert alert-block alert-info">
<b>Aide </b>
</div>

N'oubliez pas que vous pouvez retourner vers les TPs précédents si vous avez des questions sur Python, Numpy, Pandas ou Matplotlib. Gardez vos cheatsheets à proximité !

- [Data wrangling with Pandas](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Matplotlib cheatscheets](https://matplotlib.org/cheatsheets/)

<div class="alert alert-block alert-info">
<b>N'hésitez pas à écrire à vos assistants directement sur Teams en cas de question.  </b>
</div>



---


<!-- @import "[TOC]" {cmd="toc" depthFrom=1 depthTo=6 orderedList=false} -->


### Table des matières<a class="anchor"  id="toc"></a>


[Partie 1: Modèle à base de règles](#Part1)

[- 1.1 Analyse exploratoire](#Part11)

[- 1.2 Boxplots, pairplot et choix des variables](#Part12)     ---    [20 points](#Part112pts)

[- 1.3 Création du modèle à base de règles](#Part13)

[- 1.4 Matrice de confusion](#Part14)     ---    [16 points](#Part134pts)

---

[Partie 2: L'algorithme des k plus proches voisins (k-NN) ](#Part2)  

[- 2.1 Implémentation](#Part21)    ----    [12 points](#Part21pts)

[- 2.2 Validation hold-out](#Part22)    ----    [10 points](#Part22pts)

[- 2.3 N-Folds Cross Validation](#Part23)    ----    [10 points](#Part23pts)

[- 2.4 Conclusion](#Part24)    ----    [8 points](#Part24pts)



---

<p style="background-color:#7ba3e3;padding:10px"><font size="6"><b></b></font></p>

---


In [1]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from math import sqrt
import itertools

# Package scikit-learn
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Pour k-NN
from collections import Counter, defaultdict

# Pour validation hold-out

import random
from random import randrange
random.seed(10)

---


## Partie 1 - Modèle à base de règles<a class="anchor"  id="Part1"></a>

[Table des matières](#toc)

[- 1.1 Analyse exploratoire](#Part11)

[- 1.2 Boxplots, pairplot et choix des variables](#Part12)     ---    [20 points](#Part112pts)

[- 1.3 Création du modèle à base de règles](#Part13)

[- 1.4 Matrice de confusion](#Part14)     ---    [16 points](#Part134pts)

Dans cette partie, nous allons créer un modèle de classification simple. 


---
<a class="anchor"  id="Part11"></a>
### 1.1 Analyse exploratoire

[Début partie 1](#Part1) --- [Table des matières](#toc)

Nous allons utiliser la base de données «Wine Data Set » disponible sur le dépôt de bases de données maintenu par l’Université de Californie à Irvine (UCI). Le lien est celui-ci : [http://archive.ics.uci.edu/ml/datasets/Wine?Quality].

Créez un DataFrame à partir du fichier **wine.data** et des noms de colonnes fournis, puis explorez rapidement les données. Ouvrez rapidement le fichier et observez les données pour voir leur structure et mieux comprendre la suite et posez-vous ces questions (pas besoin de noter une réponse): 

- *De quel manière sont stockées les données ?* 
- *Pourquoi on nous donne une liste de noms de colonne ?*

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Exercice </b></font> Chargez les données comme indiqué ci-dessous. Si vous le voulez vous pouvez directement charger ces données depuis Scikit-learn, à vous de trouver comment faire.</p>
(2 points)



Notes:

- Utilisez la fonction read_csv() pour lire le fichier **wine.data** et spécifiez le paramètre *names* en lui passant la liste donnée ci-dessous.
- La fonction [_unique()_](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) peut vous être utile !

In [2]:
from sklearn.datasets import load_wine

# Noms de colonnes 

headers = ['Class', 'Alcool', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 
           'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 
           'Hue', 'OD280/OD315 of diluted wines', 'Proline']

# Créez le dataset et nommez le "wines"
# Ajoutez votre code ici.

wines = pd.read_csv('wine.data', header=None, names=headers)
print(wines.describe())
print("\nClasses uniques :")
print(wines["Class"].unique())

sklearn_wine_data = load_wine()
df_sklearn = pd.DataFrame(data=sklearn_wine_data.data, columns=sklearn_wine_data.feature_names)
df_sklearn["Class"] = sklearn_wine_data.target
print(df_sklearn.head())



            Class      Alcool  Malic acid         Ash  Alcalinity of ash  \
count  178.000000  178.000000  178.000000  178.000000         178.000000   
mean     1.938202   13.000618    2.336348    2.366517          19.494944   
std      0.775035    0.811827    1.117146    0.274344           3.339564   
min      1.000000   11.030000    0.740000    1.360000          10.600000   
25%      1.000000   12.362500    1.602500    2.210000          17.200000   
50%      2.000000   13.050000    1.865000    2.360000          19.500000   
75%      3.000000   13.677500    3.082500    2.557500          21.500000   
max      3.000000   14.830000    5.800000    3.230000          30.000000   

        Magnesium  Total phenols  Flavanoids  Nonflavanoid phenols  \
count  178.000000     178.000000  178.000000            178.000000   
mean    99.741573       2.295112    2.029270              0.361854   
std     14.282484       0.625851    0.998859              0.124453   
min     70.000000       0.980000   

Explorez rapidement les données avec les fonctions habituelles pour récolter des informations (p.ex., nombre d’observations, de classes, d’attributs, statistiques des attributs, données manquantes). 

In [3]:
# Ajoutez votre code ici. Ajoutez des cellules si nécessaire. 

wines = pd.read_csv('wine.data', header=None, names=headers)
print(wines.describe())
print("\nClasses uniques :")
print(wines["Class"].unique())

sklearn_wine_data = load_wine()
df_sklearn = pd.DataFrame(data=sklearn_wine_data.data, columns=sklearn_wine_data.feature_names)
df_sklearn["Class"] = sklearn_wine_data.target
print(df_sklearn.head())

            Class      Alcool  Malic acid         Ash  Alcalinity of ash  \
count  178.000000  178.000000  178.000000  178.000000         178.000000   
mean     1.938202   13.000618    2.336348    2.366517          19.494944   
std      0.775035    0.811827    1.117146    0.274344           3.339564   
min      1.000000   11.030000    0.740000    1.360000          10.600000   
25%      1.000000   12.362500    1.602500    2.210000          17.200000   
50%      2.000000   13.050000    1.865000    2.360000          19.500000   
75%      3.000000   13.677500    3.082500    2.557500          21.500000   
max      3.000000   14.830000    5.800000    3.230000          30.000000   

        Magnesium  Total phenols  Flavanoids  Nonflavanoid phenols  \
count  178.000000     178.000000  178.000000            178.000000   
mean    99.741573       2.295112    2.029270              0.361854   
std     14.282484       0.625851    0.998859              0.124453   
min     70.000000       0.980000   

---

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> La colonne "Class" contient notre variable dépendante (variable de sortie / output). Combien de classes différentes y a-t-il et à quoi correspondent-elles ? A quelle position se trouve cette colonne dans le DataFrame ?  </p> (2 points)

*Réponses:*



---
<a name="Part12"></a>
### 1.2 Boxplots, pairplot et choix des variables

[Début partie 1](#Part1) --- [Table des matières](#toc)


Maintenant que nous connaissons mieux les données, nous pouvons visualiser les différences entre classes afin de pouvoir créer des règles de classification. Le but, est de trouver des valeurs de variables permettant de séparer les différentes classes du mieux possible afin de pouvoir classifier de nouveaux vins. 

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Exercice </b></font> Créez un boxplot pour chaque variables, groupées par Class.</p>
(4 points)

Pour faire ceci, complétez le code ci-dessous en vous basant sur le pseudo code donné en commentaire entre les "------". N'utilisez que 2 lignes: une pour la boucle et une pour le graphique.

- Aide: Pour obtenir les noms de colonnes utilisez *wines.columns*. Pour obtenir i dans la boucle, en plus des noms de colonne, utilisez la fonction *enumerate*.
- Aide: Utilisez le [slicing](https://stackoverflow.com/questions/509211/understanding-slicing) pour obtenir tous les noms de colonne sauf Class (qui se trouve à une extrémité de la liste). 
- Aide: Utilisez la fonction [sns.boxplot de Seaborn](https://seaborn.pydata.org/generated/seaborn.boxplot.html) pour un résultat élégant très coloré, ou la fonction [pandas.DataFrame.boxplot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.boxplot.html) pour un résultat épuré en toute simplicité.

In [ ]:
fig, axs = plt.subplots(2, 7, figsize=(20, 10)) # On spécifie le nombre de lignes et de colonnes de notre figure 
axs = axs.flatten() # -> 1D

#----------
# Pour i et chaque nom de colonne sauf class
    # Boxplot avec pour x la Class, et pour y la variable. Utilisez ax=axs[i] pour insérer le plot dans la figure. !
#----------  

for i, col in enumerate(wines.columns[1:]):
    sns.boxplot(x="Class", y=col, data=wines, ax=axs[i]) 


fig.suptitle('Boxplot of the features by label', fontsize=14)    
fig.tight_layout(rect=[0, 0.03, 1, 0.98])
fig.delaxes(axs[13])

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Exercice </b></font> Créez un Pairplot à l'aide de la librairie Seaborn</p>
(4 points)

****

Pour les données avec "peu" de variables, comme dans notre cas, un pairplot peut permettre d'obtenir une excellente vue d'ensemble. Cependant, la génération du pairplot peut prendre un certain temps. Pour éviter de prendre du temps à générer et re-générer le pairplot, vous pouvez commenter votre code une fois que vous avez analysé le graphique.

Note: Vous pouvez désactiver les Warnings s'il y en a. 

In [ ]:
# Pairplot avec Seaborn - Votre code

# Désactiver les warnings s'il y en a
import warnings
warnings.filterwarnings("ignore")

sns.pairplot(wines, hue="Class", diag_kind="kde", palette="tab10")
plt.show()




---

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Quels éléments du pairplot permettent de rapidement distinguer les différences entre les 3 classes ? </p> (2 points)

*Réponse:*

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> D'après les boxplots, quelles variables permettraient le mieux de distinguer/séparer les trois classes ? Et de distinguer une classe des deux autres ? Donnez 5 variables (en tout) soit pour séparer les 3 classes soit une des deux autres, ainsi que les valeurs approximatives de séparration. Justifiez et discutez votre choix. </p> (6 points)

<p></p>
<div style="background-color:#EEEEEE;padding:3px">Note: un modèle avec des règles simples peut permettre, par exemple, de distinguer deux classes l'une de l'autre. Par exemple pour distinguer un chat d'un chien sur des valeurs numérique on pourrait avoir les règles suivantes: 

- Si $souplesse < 2$ et $poids > 10$ -> c'est un chien
- Sinon -> c'est un chat. 

Pour pouvoir établir ces règles, il faut donc avoir une vue sur les différentes variables. Ce n'est pas parfait mais le modèle reste simple. Il fonctionnera dans une bonne partie des cas mais on peut toujours tomber sur un Chihuahua souple de moins de 5 kilos par exemple. 

*Réponse:*

<a name="Part112pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 1.1 - 1.2: </b> Points obtenus: /20
</div>

Remarques de l'assistant: 
- Load data /2
- Q /2
- Plots /8
- Q /2
- Q /6


[Début partie 1](#Part1) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>

<a name="Part13"></a>
### 1.3 Création du modèle à base de règles

[Début partie 1](#Part1) --- [Table des matières](#toc)



Suivez la structure du code ci-après pour implémenter un modèle à base de règles. Complétez les parties manquantes et écrivez les fonctions demandées. **Lisez bien les commentaires afin de bien comprendre la logique implémentée**.

#### 1.3.1 Définition des règles

Dans cette partie vous allez écrire les règles permettant de distinguer les trois classes (Note) à l'aide des variables `Alcool`, `Color intensity` et `Flavanoids`. Utilisez pour cela les valeurs que vous avez trouvées au TP3 ou servez vous des graphiques ci-dessus pour les déterminer. 

Servez-vous de la fonction donnée en exemple pour créer les fonctions suivantes (**respectez les noms !**): 

- *class_by_alcool()*: utilise la variable Alcool uniquement. 
- *class_by_flavanoids()*: utilise la variable Flavanoids uniquement.
- *class_by_color_intensity()*: utilise la variable Color intensity uniquement
- *class_by_flav_alcool()*: utilise les variables Flavanoids et Alcool
- *class_by_color_alcool()*: utilise les variables Color intensity et Alcool
- *class_by_color_flav()*: utilise les variables Color intensity et Flavanoids


In [ ]:
# Les fonctions suivantes seront appliquées au DataFrame à l'aide de la fonction apply(). 
# Le DataFrame sera donc passé en paramètres et les opérations effectuées sur toutes les observations. 


# ------------------------------------------------------------------
# Définition des règles
# ------------------------------------------------------------------
'''
def class_by_XXXXXX(df): # ou class_by_XXXXXX_YYYYYY(df):
    """
    Retourne la classe (1, 2 ou 3) selon la variable XXXXXX (et YYYYYY -> en ayant accès à tout le dataframe, on peut
    aussi tester une seconde variable ou plus).
    """
    if(df["XXXXXX"] > 42 and....):
        return 1
    elif(df["YYYYYY"] > 42):
        return 2
    else:
        return 3
'''

# Fonction pour distinguer les classes uniquement avec la variable Alcool
def class_by_alcool(df):
    if df["Alcool"] > 13.5:
        return 1
    elif 12.5 < df["Alcool"] <= 13.5:
        return 2
    else:
        return 3

# Fonction pour distinguer les classes uniquement avec la variable Flavanoids
def class_by_flavanoids(df):
    if df["Flavanoids"] > 2.5:
        return 1
    elif 1.5 < df["Flavanoids"] <= 2.5:
        return 2
    else:
        return 3

# Fonction pour distinguer les classes uniquement avec la variable Color intensity
def class_by_color_intensity(df):
    if df["Color intensity"] > 7:
        return 3
    elif 5 < df["Color intensity"] <= 7:
        return 2
    else:
        return 1

# Fonction pour distinguer les classes avec les variables Flavanoids et Alcool
def class_by_flav_alcool(df):
    if df["Flavanoids"] > 2.5 and df["Alcool"] > 13.5:
        return 1
    elif df["Flavanoids"] <= 2.5 and 12.5 < df["Alcool"] <= 13.5:
        return 2
    else:
        return 3

# Fonction pour distinguer les classes avec les variables Color intensity et Alcool
def class_by_color_alcool(df):
    if df["Color intensity"] > 7 and df["Alcool"] < 12.5:
        return 3
    elif 5 < df["Color intensity"] <= 7 and 12.5 < df["Alcool"] <= 13.5:
        return 2
    else:
        return 1

# Fonction pour distinguer les classes avec les variables Color intensity et Flavanoids
def class_by_color_flav(df):
    if df["Color intensity"] > 7 and df["Flavanoids"] <= 1.5:
        return 3
    elif 5 < df["Color intensity"] <= 7 and 1.5 < df["Flavanoids"] <= 2.5:
        return 2
    else:
        return 1



#### 1.3.2 Prédictions

Pour chacune des règles créées ci-dessus, ajoutez une colonne dans df, une copie du DataFrame wines. Utilisez des noms parlants comme "pred_alcool" ou "pred_alcool_flav" afin que l'on comprenne bien de quelle prédiction il s'agit. Un exemple de l'utilisation de la fonction apply() est donné en commentaire. 


In [ ]:
# ------------------------------------------------------------------
# Creation de la / des colonnes de prédiction 
# ------------------------------------------------------------------

# Copie du DataFrame pour ne pas modifier l'original accidentellement. 
df = wines.copy()


# Créez une nouvelle colonne qui contient le résultat de la fonction class_by_YYYYYY
# Faites le pour chaque fonction créées ci-dessus. 

# Ajout des colonnes de prédictions pour chaque règle définie
df["pred_alcool"] = df.apply(class_by_alcool, axis=1)
df["pred_flavanoids"] = df.apply(class_by_flavanoids, axis=1)
df["pred_color_intensity"] = df.apply(class_by_color_intensity, axis=1)
df["pred_flav_alcool"] = df.apply(class_by_flav_alcool, axis=1)
df["pred_color_alcool"] = df.apply(class_by_color_alcool, axis=1)
df["pred_color_flav"] = df.apply(class_by_color_flav, axis=1)
print(df.head())

# Exemple: df["pred_NNN"] = df.apply(class_by_YYYYYY, axis=1)



<a name="Part14"></a>
### 1.4 Matrice de confusion

[Début partie 1](#Part1) --- [Table des matières](#toc)




Affichez les scores (Accuracy) et matrices de confusion pour chaque règles. Pour cela, on utilise les fonctions **accuracy_score()**, **confusion_matrix()** et **ConfusionMatrixDisplay()** fournies par ScikitLearn et importée en amont de ce Notebook. 

Inspirez-vous du code partiellement donné en exemple ci-dessous. Vos résultats doivent cependant être présentés de manière à ce qu'on puisse savoir à quelle règle correspond quelle accuracy /  matrice de confusion. 

In [ ]:
# ------------------------------------------------------------------
# Pour chaque modèle (6x), affichez le score (Accuracy) et la matrice de confusion (code ci-dessous)
# ------------------------------------------------------------------

# Vous pouvez écrire une boucle pour afficher le score et la matrice à la suite ou dans des cellules séparées. 
# Libre à vous de choisir la solution qui vous plait le plus !

'''
score = accuracy_score(...)
print(f"Accuracy: {score}")

cm = confusion_matrix(...)
disp = ConfusionMatrixDisplay(...)
disp.plot()
plt.show()
'''

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Liste des colonnes de prédictions
pred_columns = [
    "pred_alcool", 
    "pred_flavanoids", 
    "pred_color_intensity", 
    "pred_flav_alcool", 
    "pred_color_alcool", 
    "pred_color_flav"
]

# Boucle pour calculer et afficher accuracy et matrices de confusion
for col in pred_columns:
    print(f"### Résultats pour {col} ###")
    
    # Calcul de l'accuracy
    score = accuracy_score(df["Class"], df[col])
    print(f"Accuracy: {score}")
    
    # Calcul de la matrice de confusion
    cm = confusion_matrix(df["Class"], df[col])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1, 2, 3])
    
    # Affichage de la matrice
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Matrice de confusion pour {col}")
    plt.show()


<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Laquelle de vos règles donne le meilleur résultat ? Y a-t-il une classe plus difficile à prédir qu'une autre ?</p>

*Réponses:*

<a name="Part134pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 1.3 - 1.4: </b> Points obtenus: /16
</div>

Remarques de l'assistant: 
- Règles (6) et prédictions (4 pts): /10
- Matrices de confusion: /4
- Question: /2


[Début partie 1](#Part1) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>

---

<p style="background-color:#7ba3e3;padding:10px"><font size="6"><b></b></font></p>

---

<a name="Part2"></a>

## Partie 2 - L'algorithme des k plus proches voisins (k-NN) 

[Table des matières](#toc)

Dans cet exercice, nous allons utiliser KNN pour traiter le problème de classification des vins.  


[- 2.1 Implémentation](#Part21)    ----    [12 points](#Part21pts)

[- 2.2 Validation hold-out](#Part22)    ----    [10 points](#Part22pts)

[- 2.3 N-Folds Cross Validation](#Part23)    ----    [10 points](#Part23pts)

[- 2.4 Conclusion](#Part24)    ----    [8 points](#Part24pts)


<a name="Part21"></a>
### 2.1 Implémentation et préparration des données
[Table des matières](#toc)

Le code suivant est tiré du cours. Complétez les [docstrings](https://www.datacamp.com/tutorial/docstrings-python#multi-line-docstring-multi) de manière à indiquer ce que fait la fonction, quels sont les paramètres ainsi que la valeur de retour. Ajoutez en plus des commentaires sur les parties complexes afin de montrer que vous avez compris l'implémentation de KNN. 


In [ ]:
# Code du cours - TODO Complétez les docstrings 


def train_test_split(dataset, split=0.60):
    """
    Divise un dataset en deux sous-ensembles : un ensemble d'entraînement et un ensemble de test.
    
    Args:
        dataset (list): Le dataset initial sous forme de liste.
        split (float): Le ratio d'entraînement (par défaut 0.60). 
                       Par exemple, 0.60 signifie 60% des données seront utilisées pour l'entraînement.

    Returns:
        tuple: Une paire (train, test), où :
               - train (list) est l'ensemble d'entraînement.
               - test (list) est l'ensemble de test.
    
    """
    train = list()
    train_size = split * len(dataset)
    test = list(dataset)
    while len(train) < train_size:
        index = randrange(len(test))
        train.append(test.pop(index))
    return train, test


def euclidean_distance(row1, row2):
    """
    Calcule la distance euclidienne entre deux vecteurs (lignes du dataset).

    Args:
        row1 (list): La première ligne (vecteur) sous forme de liste.
        row2 (list): La deuxième ligne (vecteur) sous forme de liste.

    Returns:
        float: La distance euclidienne entre les deux vecteurs.
    """

    distance = 0.0
    for i in range(1, len(row1)): # All columns except the first one (Note) - Wrong in course's PDF
        distance += (row1[i] - row2[i])**2
    
    return sqrt(distance)


def get_neighbors(train, test_row, num_neighbors):
    """
    Identifie les 'k' voisins les plus proches d'une ligne donnée dans l'ensemble d'entraînement.

    Args:
        train (list): L'ensemble d'entraînement sous forme de liste.
        test_row (list): Une ligne de l'ensemble de test pour laquelle on cherche les voisins.
        num_neighbors (int): Le nombre de voisins à considérer.

    Returns:
        list: Les 'k' voisins les plus proches sous forme de liste.
    """
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors


def predict_classification(train: pd.DataFrame, test_row, num_neighbors):
    """
    Prédit la classe d'une ligne de test basée sur les classes des 'k' voisins les plus proches.

    Args:
        train (list): L'ensemble d'entraînement sous forme de liste.
        test_row (list): Une ligne de test pour laquelle on veut prédire la classe.
        num_neighbors (int): Le nombre de voisins à considérer.

    Returns:
        int/str: La classe prédite (dépend des données).
    """
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[0] for row in neighbors]
    #print(f"Output values classification: {output_values}")
    prediction = max(set(output_values), key=output_values.count)
    return prediction


def accuracy_metric(actual: list[float], predicted: list[float]):
    """
    Calcule le pourcentage de précision des prédictions.

    Args:
        actual (list): Liste des classes réelles.
        predicted (list): Liste des classes prédites.

    Returns:
        float: Pourcentage de précision (valeur entre 0 et 100).
    """
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


def cross_validation_split(dataset, n_folds):
    """
     Divise le dataset en 'n_folds' sous-ensembles pour effectuer une validation croisée.

    Args:
        dataset (list): Le dataset initial sous forme de liste.
        n_folds (int): Le nombre de sous-ensembles (folds) à créer.

    Returns:
        list: Une liste contenant 'n_folds' sous-ensembles, chacun étant une partie du dataset.

    """
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


Création de deux datasets: 
- un avec les données brutes
- un avec les données normalisées

**Lisez sans modifier le code ci-dessous pour en comprendre le sens et répondez aux questions.**

In [ ]:
# Préparation des datasets - rien à modifier

#=====================================================================
# Préparation des données brutes 
#=====================================================================

# Conversion en liste de listes au lieu de DataFrame
data = wines.values.tolist()

#=====================================================================
# Préparation des données normalisées
#=====================================================================


# Différents moyens de standardiser les données mise à disposition par Scikit-learn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Nous allons utiliser le StandardScaler, qui va normaliser les données avec une moyenne de 0 et un écart-type de 1
scaler = StandardScaler()
scaler_string = "StandardScaler" # pour l'affichage dans le titre des plots si nécessaire


# Créer une copie et prendre une liste des colonnes des données indépendantes (entrées)
df_normalized = wines.copy()
cols = wines.columns[wines.columns != 'Note']

# Remplacer les colonnes des données par les données transformées
df_normalized[cols] = scaler.fit_transform(df_normalized[cols])

# conversion en liste de listes pour k-NN
data_normalized = df_normalized.values.tolist()

# K's à tester
Ks = [1, 2, 3, 5, 7, 10]

**Ci-dessous, affichez les boxplots des données normalisées, comme pour les données du [point 1.2](#Part12).**

In [ ]:
# Normaliser les données (exclure 'Class')

scaler = MinMaxScaler()

data_normalized = scaler.fit_transform(df_normalized.drop(columns=['Class']))

# Créer un DataFrame normalisé en ajoutant les noms de colonnes

df_normalized_boxplot = pd.DataFrame(data_normalized, columns=df_normalized.columns[:-1])

# Créer les boxplots

plt.figure(figsize=(12, 6))

sns.boxplot(data=df_normalized_boxplot, palette="Set2")

plt.title("Boxplots des données normalisées (MinMaxScaler)", fontsize=16)

plt.xlabel("Variables", fontsize=12)

plt.ylabel("Valeurs normalisées", fontsize=12)

plt.xticks(rotation=45)  # Rotation des étiquettes pour une meilleure lisibilité

plt.grid(axis='y', linestyle='--', alpha=0.7)

# Affichage

plt.tight_layout()
plt.show()


<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Quelle est la différence entre un commentaire et une docstring ? </p> (1 point)

*Réponse:* Un commentaire est une note incluse dans le code pour expliquer son fonctionnement et une docstring est une chaîne de caractères qui documente un module, une classe, une méthode ou une fonction. Contrairement aux commentaires, les docstrings sont conçues pour être accessibles dynamiquement via des outils comme __doc__

Source : https://www.docstring.fr/glossaire/docstring/

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Qu'est-ce que la normalisation des données ? </p> (1 point)

Décrivez le concept en quelques mots. Attention, les termes "Normalisation" et "Standardisation" sout souvents mélangés mais le concept général reste le même !


*Réponse:* La normalisation des données est une technique utilisée en prétraitement des données pour transformer les valeurs des variables dans une plage commune ou comparable, afin de faciliter l'analyse et d'améliorer les performances des algorithmes d'apprentissage automatique.



<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Quelles sont les différences entre les StandardScaler, MinMaxScaler et RobustScaler ?</p> (3 points)

Note: Aidez-vous de [la doc](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) ou d'autres ressources pour répondre. 

*Réponse:* 

StandardScaler :

- Centrage sur une moyenne de 0 et écart-type de 1.
- Sensible aux outliers (valeurs extrêmes).

MinMaxScaler :

- Mise à l'échelle entre 0 et 1 (ou autre intervalle choisi).
- Sensible aux outliers car basé sur les minimum et maximum.

RobustScaler :

- Utilise la médiane et l'IQR (Intervalle Interquartile).
- Résistant aux outliers.

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Quelles différences observez-vous entre les boxplots des données originales du point 1.2 par rapport aux boxplots avec les données normalisées ?</p>(1 points)

Voir [point 1.2](#Part12).

*Réponse:* 

- Échelle uniforme : Les données normalisées ont toutes leurs valeurs entre 0 et 1, rendant les comparaisons entre variables plus claires. Dans les données originales, les échelles varient fortement (par ex. proline vs hue).

- Visibilité des variables : Dans les données normalisées, les variables moins dominantes (comme hue) deviennent aussi visibles que les autres.

- Analyse simplifiée : La normalisation met en évidence les différences entre classes pour toutes les variables, facilitant l'identification des caractéristiques discriminantes.

<a name="Part21pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 2.1: </b> Points obtenus: /12
</div>

Remarques de l'assistant: 

- docstrings: /6
- Questions: /6


[Début partie 2](#Part2) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>



<a name="Part22"></a>
### 2.2 Validation hold-out
[Table des matières](#toc)

Nous allons ici utiliser la méthode "Validation Hold-Out" avec k-NN sur nos données. 

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Exercice </b></font> Complétez le code ci-dessous en suivant les indications en commentaire. Respectez le nombre de lignes indiqué.</p> (6 points)



Utilisez en premier le dataset "***data***" puis une fois votre code fonctionnel, ajoutez un niveau de boucle et effectuez la même opération en plus pour le dataset avec les données normalisées: "***data_normalized***". 



*Supprimez aussi le print() dans la boucle qui est là pour que le code fonctionne avant que vous l'ayez complété.* 

Aide: 
- Vous devez modifier différentes variables, comme par exemple ajouter une dimention à la variable qui contient les résultats. Pour cela on vous conseille de créer deux dictionnaires (comme c'est déjà fait), un pour les données originales et un pour les données normalisées puis de créer une liste "results" contenant vos deux dictionnaires de résultats. N'oubliez pas de modifier le titre des graphiques pour plus de clareté (indiquez quel données sont utilisées). 
- Si vous voulez boucler sur les éléments d'une liste (contenant par exemple deux datasets) vous pouvez utiliser la fonction *enumerate()* qui retourne chaque élément en plus d'un indice (qu'on appelera j ou l par exemple)
- Pour adapter le titre de vos graphiques, vous pouvez simplement créer une liste avec deux titres et utiliser votre variable de boucle pour choisir le bon titre





In [ ]:
# Complétez le code ci-dessous. Attention, les données sont dans "data"/"data_normalized" et non dans "wines":


# Nous voulons 10 répétitions avec la méthode validation hold-out
N_REPETITIONS = 10

# Nous allons stocker les résultats dans un dictionnaire, avec pour chaque k une liste d'accuracy. 
results = defaultdict(list)

# Liste des datasets et des titres associés
datasets = [data, data_normalized]
titles = ['Données Originales', 'Données Normalisées']

# Boucle principale pour chaque dataset
for j, dataset in enumerate(datasets):
    # Pour chaque validation hold-out (test et train set différents) on test chaque k
    for i in range(N_REPETITIONS):
        for k in Ks:
            # Vous pouvez supprimer cette ligne une fois votre code fonctionnel
            # Pour que le code tourne, la boucle ne doit pas être vide ;-)  
            
            # Faire le train-test split avec split = 0.8. 
            # La fonction retourne deux éléments, utilisez le unpacking pour les extraire (train, test = fct())
            # - complétez - 1 ligne
            train, test = train_test_split(dataset,split=0.8)
    
            # Aide: utilisez la compréhension de liste:  prédictions = [prediction(...) for ... in ...] 
            #                                                                           pour chaque observation de test
            #       Vous devez générer une liste de prédictions (1, 2 ou 3), que vous comparerez en dessous avec 
            #         les valeurs justes pour obtenir l'accuracy.
            
            # - complétez - 1 ligne
            predictions = [predict_classification(train, row, k) for row in test]
    
            # Joindre à results[k] la liste des accuracies (obtenu avec la fonction accuracy_metric() ci dessus)
            # Aide: utilisez la méthode append() pour joindre l'accuracy aux résultats
            # Aide: utilisez la méthode accuracy_metric() pour obtenir l'accuracy
            # Aide: Vous avez une liste de prédictions et un dataframe original (test)
            #       utilisez la compréhension de liste pour créer la liste des données originales:
            #         [colone "note" pour chaque ligne des tests]
            
            # - complétez - 1 ligne (si vous y arrivez, sinon plus)
            
            results[k].append(accuracy_metric([row[0] for row in test], predictions))
    
    
    # Une fois le code fonctionnel, vous devrez bouger le graphique à l'intérieur de la boucle 
    # N'oubliez pas de changer le titre selon les données utilisées !
    
    _, ax = plt.subplots(1, 1, figsize=(16, 8))
    
    means = [np.mean(v) for v in results.values()]
    stds = [np.std(v) for v in results.values()]
    
    ax.errorbar(results.keys(), means, stds, linestyle='None', marker='D', color='darkgreen')
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_xlabel('K', fontsize=12)
    ax.set_title(f'Performance of classification with various K using K-NN ({titles[j]})', fontsize=14)
    ax.grid(axis='y', linestyle='--', alpha=0.4, lw=0.5)

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Pour quelle valeur(s) de k le modèle est-il le plus efficace sans et avec les données normalisées ? Une fois les données normalisées, quelle différence observe-t-on ? Justifiez.</p>
(2 points)

Note: Les résultats peuvent varier d'une exécution à l'autre. Indiquez la tendance générale que vous observez.



*Réponse:*



<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Pourquoi les résultats varient d'une exécution à une autre ? Décrivez le fonctionnement de ce que vous venez de faire (point 2.1) en quelques phrases.</p> (2 points)

*Réponse:*

<a name="Part22pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 2.2: </b> Points obtenus: /10
</div>

Remarques de l'assistant: 

- Exercice code: /6
- Q /4


[Début partie 2](#Part2) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>


<a name="Part23"></a>
### 2.3 N-Folds cross validation
[Table des matières](#toc)


Si vous voulez vous raffraichir sur ce qu'est la N-Fold Cross Validation, voici [une courte vidéo de StatQuest](https://www.youtube.com/watch?v=fSytzGwwBVw) (ou [une autre en français](https://www.youtube.com/watch?v=xWgGOHiROmc)).

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Exercice </b></font> Complétez le code ci-dessous en suivant les indications en commentaire, cette fois en utilisant la N-fold cross validation.</p>(6 points)

Note: 
- Aussi appelée k-fold cross validation dans la litérature, mais là on veut pas confondre avec notre k de k-NN. 

Utilisez en premier le dataset "***data***" puis une fois votre code fonctionnel, ajoutez un niveau de boucle et ajoutez le dataset avec les données normalisées: "***data_normalized***" comme au [point 2.2](#Part22). N'oubliez pas de modifier les parties nécessaires ainsi que le titre des graphiques pour que le résultat soit lisible. 

*Supprimez aussi le print() dans la boucle qui est là pour que le code fonctionne avant que vous l'ayez complété.* 

In [ ]:
results_nfold = defaultdict(list)

# Les données sont divisées en plusieurs folds 
folds = cross_validation_split(data, n_folds=5)


# Boucle sur les datasets
for j, dataset in enumerate(datasets):  
    for k in Ks:
        
        # Listes qui vont contenir les valeurs prédites et les vraies valeurs
        predictions = []
        actuals = []
    
        for test_i in range(len(folds)):
            # print(f"----- Fold = {test_i } - k = {k} -----") 
            
            # train = tout sauf le fold i, qui est utilisé pour tester.
            # Permet de créer une liste à partir des éléments de folds, y compris si c'est vide. 
            train = list(itertools.chain.from_iterable(folds[:test_i] + folds[test_i+1:]))
                
            # test = Fold courrant, utilisé pour tester
            # - complétez - 1 ligne
            test = folds[test_i]
            
            
            # Effectue la prédiction et l'ajoute au tableau prédictions[] pour chaque element dans test
            # predictions += [...fonction pour faire la classification définie en début de point 2.... for....in ....]
            # - complétez - 1 ligne
            predictions += [predict_classification(train, row, k) for row in test]
                    
            
            # Remplit la liste "actual" avec les valeurs réelles (actuals += ... )
            # - complétez - 1 ligne
            actuals += [row[0] for row in test] 
        
        # Mettez le résultat de "accuracy_metric(...,...)" dans results_nfold[k]
        # - complétez - 1 ligne
        accuracy = accuracy_metric(actuals, predictions)
        results_nfold[k].append(accuracy) 
        
        print(f'Performance with k={k}: {results_nfold[k]}')
        
    _, ax = plt.subplots(1, 1, figsize=(16, 8))
    
    
    # Une fois le code fonctionnel pour un dataset, vous devrez bouger les graphiques à l'intérieur de la boucle 
    # comme au point 2.2. 
    
    ax.plot(results_nfold.keys(), results_nfold.values(), linestyle='None', marker='D')
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_xlabel('K', fontsize=12)
    ax.set_title(f'Performance of classification with various K using K-NN (K-fold cross-validation)({titles[j]})', fontsize=14)
    ax.grid(axis='y', linestyle='--', alpha=0.4, lw=0.5)

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Quelle différence dans les code des points 2.2 (validation hold-out et 2.3 (N-Fold cross validation) fait que nous n'avons qu'un seul point par k et non une moyenne et écart-type ? </p>(2 points)

*Réponse:*
La différence réside dans le fait que pour la validation hold-out, on obtient un seul score d'accuracy par valeur de k. Alors que pour la validation croisée (N folds), on obtient une moyenne et un écart-type des résultats de précision pour chaque valeur de k, en utilisant les résultats de tous les folds.

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Les résultats varient-ils d'une exécution à l'autre ? Décrivez le fonctionnement de ce que vous venez de faire (point 2.2) en quelques phrases. </p>(2 points)

*Réponse:*
Dans le point 2.2 (validation hold-out), les données sont divisées en deux ensembles : un pour l'entraînement et un autre pour le test. Le modèle k-NN est ensuite entraîné sur l'ensemble d'entraînement et testé sur l'ensemble de test pour chaque valeur de k. L'accuracy, c'est-à-dire le taux de bonnes prédictions, est calculée et enregistrée pour chaque valeur de k. Cette procédure est répétée pour plusieurs valeurs de k afin d'identifier la meilleure configuration. Comme l'échantillonnage est aléatoire, les résultats peuvent varier légèrement d'une exécution à l'autre.



<a name="Part23pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 2.3: </b> Points obtenus: /10
</div>

Remarques de l'assistant: 
- code /6
- Q /4


[Début partie 2](#Part2) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>



<a name="Part24"></a>
### 2.4 Conclusion
[Début partie 2](#Part2) --- [Table des matières](#toc)



Répondez aux questions suivantes **en faisant des phrases**. 

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font> Quels sont les principaux avantages de l'utilisation des modèles basés sur des règles par rapport à d'autres algorithmes d'apprentissage automatique comme le KNN ? </p>(2 points)

*Réponse:*
Les modèles basés sur des règles, comme les arbres de décision, sont plus interprétables, rapides en prédiction, et moins sensibles au bruit que le KNN. Ils gèrent mieux les variables catégorielles et modélisent facilement les relations non linéaires, contrairement au KNN qui est plus lent et dépend des distances, rendant l'interprétation plus complexe.

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Comparez et contrastez les techniques de validation hold-out et de N-Fold Cross Validation dans le contexte de l'évaluation des performances d'un classifieur KNN. </p>(4 points)

*Réponse:*
La validation hold-out divise les données en un ensemble d'entraînement et un ensemble de test, mais les résultats peuvent varier selon la partition. La N-Fold Cross Validation utilise plusieurs sous-ensembles pour tester chaque donnée, offrant une évaluation plus robuste et moins biaisée, mais au coût de temps de calcul plus élevé.

<p style="background-color:#AFEEEE;padding:3px"><font size="4"><b>Q </b></font>Comment la nature "lazy learning" du KNN impacte-t-elle ses performances et son efficacité computationnelle par rapport à d'autres algorithmes ? </p>(2 points)

*Réponse:*
La nature "lazy learning" du KNN signifie qu'il n'apprend pas pendant l'entraînement, mais compare chaque nouvelle instance avec toutes les données d'entraînement lors de la prédiction, ce qui le rend précis mais computationalement coûteux. Contrairement à d'autres algorithmes comme les arbres de décision, qui apprennent un modèle et prédisent plus rapidement.

<a name="Part24pts"></a>
<div class="alert alert-block alert-info">
<b>Corrections Partie 2.4: </b> Points obtenus: /8
</div>

Remarques de l'assistant: 


[Début partie 2](#Part2) --- [Table des matières](#toc)

<p style="background-color:#003c9c;padding:3px"><font size="4"><b></b></font></p>

Fin